# Datenanaylse des RTzW'95
In diesem Notenoob werden die Ergebnisse der Datenanalyse festgehalten.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re

---
## Erstellen des Dataframes

In [9]:
# Create the DataFrame
# "Fassung","Artikel","Inhalt" are strings, 
# "Datum" is a datetime and "Anzahl_woerter" is an integer.
# "Fassung" and "Artikel" or "Datum" and "Artikel" are unique identifiers.
df = pd.DataFrame({
    "Fassung": [],
    "Datum": [],
    "Artikel": [],
    "Inhalt": [],
    "Anzahl_woerter": [],
})
print("DataFrame created with columns: Fassung, Datum, Artikel, Inhalt, Anzahl_woerter")
df.head()

DataFrame created with columns: Fassung, Datum, Artikel, Inhalt, Anzahl_woerter


,Fassung,Datum,Artikel,Inhalt,Anzahl_woerter


In [ ]:
# Datum für jede Fassung
date_dict = {
    1 : "28.04.1495",
    2 : "15.05.1495",
    3 : "01.06.1495",
    4 : "07.08.1495",
}


def fill_dataframe_from_txt(file_path):
    entries = []
    fassung = None
    artikel = None
    text_lines = []

    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()

            # Neue Fassung erkennen (sowohl "# 2. Fassung #" als auch "# 2. Fassung ---------------------------------")
            if match_fassung := re.search(r"#\s*(\d+)\.\s*Fassung", line):
                # Vorherigen Artikel speichern, falls vorhanden
                if artikel and text_lines:
                    text = " ".join(text_lines).strip()
                    entries.append(
                        {
                            "Fassung": fassung,
                            "Datum": date_dict.get(fassung, "unbekannt"),
                            "Artikel": artikel,
                            "Inhalt": text,
                            "Anzahl_Woerter": len(text.split()),
                        }
                    )
                fassung = int(match_fassung.group(1))
                artikel = None
                text_lines = []
                continue

            # Artikelanfang erkennen (z.B. 1., 12., etc.)
            if match_artikel := re.match(r"^(\d{1,2}\.)\s", line):
                # Vorherigen Artikel speichern
                if artikel and text_lines:
                    text = " ".join(text_lines).strip()
                    entries.append(
                        {
                            "Fassung": fassung,
                            "Datum": date_dict.get(fassung, "unbekannt"),
                            "Artikel": artikel,
                            "Inhalt": text,
                            "Anzahl_Woerter": len(text.split()),
                        }
                    )
                artikel = match_artikel.group(1)
                text_lines = [line]  # Starte mit dieser Zeile
            elif artikel:
                # Fortsetzung des Artikels
                text_lines.append(line)

        # Letzten Artikel speichern
        if artikel and text_lines:
            text = " ".join(text_lines).strip()
            entries.append(
                {
                    "Fassung": fassung,
                    "Datum": date_dict.get(fassung, "unbekannt"),
                    "Artikel": artikel,
                    "Inhalt": text,
                    "Anzahl_Woerter": len(text.split()),
                }
            )

    return pd.DataFrame(entries)


# Beispielhafte Verwendung:
df = fill_dataframe_from_txt("Python-OCR/OCR-Text/RKG_cleaned.txt")





In [39]:
# remove intergers at the beginning of the "Inhalt" column
def remove_integers_from_inhalt(df):
    df["Inhalt"] = df["Inhalt"].apply(lambda x: re.sub(r'^\d+\.\s*', '', x))
    return df
df = remove_integers_from_inhalt(df)

# remove dots at the end of "Artikel" column
def remove_dots_from_artikel(df):
    df["Artikel"] = df["Artikel"].apply(lambda x: re.sub(r'\.$', '', x))
    # convert "Artikel" to integer type
    df["Artikel"] = pd.to_numeric(df["Artikel"], errors='coerce').astype('Int64')
    return df
df = remove_dots_from_artikel(df)

In [40]:
df.head(10)

,Fassung,Datum,Artikel,Inhalt,Anzahl_Woerter
0,1,28.04.1495,1,"Zum ersten, das cammergericht zu besotzen mit ...",270
1,1,28.04.1495,2,Item so der urteler ainer oder mehr oder auch ...,88
2,1,28.04.1495,3,Des richters und der heysitzor aid. Item die a...,190
3,1,28.04.1495,4,Item es soll kain citacion oder ladung ausgeen...,109
4,1,28.04.1495,5,gerichtschreiber aid: Item durch den cammerric...,199
5,1,28.04.1495,6,"Der redner aid. Item die redner, so das gerich...",281
6,1,28.04.1495,7,In solcher maf sollen auch die advocaten swere...,29
7,1,28.04.1495,8,Item damit auch der gemain mann unbillicher we...,131
8,1,28.04.1495,9,Item kain partey soll mehr dann ein advocaten ...,60
9,1,28.04.1495,10,"Item ob W.n, Gff., Hh., ritterschaft oder stet...",97


In [44]:
# get the unique items in the "Fassung" column
unique_fassungen = df["Fassung"].unique()
unique_artiekel = df["Artikel"].unique()
unique_artiekel = np.sort(unique_artiekel)
print(f"Unique Artikel: {unique_artiekel}")
print(f"Unique Fassungen: {unique_fassungen}")

Unique Artikel: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
Unique Fassungen: [1 2 3 4]
